In [ ]:
from fastai.vision.all import *

learn = load_learner('model.pkl')

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import base64
from io import BytesIO

cred = credentials.Certificate('firebase_key.json')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [ ]:
doc_ref = db.collection(u'preds').add({
    u'image': u'Lovelace',
    u'prediction': 1815,
    u'time_added': firestore.SERVER_TIMESTAMP
})

In [13]:
preds_ref = db.collection(u'preds')
docs = preds_ref.stream()

for doc in docs:
    data = base64.b64decode(doc.to_dict()['image'])
    buff = BytesIO(data)
    img = Image.open(buff)
    img.show()

In [1]:
import gradio as gr
from fastai.vision.all import *
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import base64
from io import BytesIO

cred = credentials.Certificate('firebase_key.json')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

learn = load_learner('model.pkl')
names = list(map(lambda name: name.title(), learn.dls.vocab))


def classify(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue())

    pred, idx, probs = learn.predict(np.asarray(image))

    db.collection(u'preds').add({
        u'image': img_str,
        u'prediction': pred.title(),
        u'time_added': firestore.SERVER_TIMESTAMP
    })

    return dict(zip(names, map(float, probs)))


with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(label="Gambar", shape=(200, 200), type='pil')
            predict_btn = gr.Button("Klasifikasi", variant='primary')
        with gr.Column():
            chart = gr.Label(label="Hasil")

    predict_btn.click(fn=classify, inputs=image_input,
                      outputs=chart, api_name='classify_image')

demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f3279fb6af0>, 'http://127.0.0.1:7860/', None)